LLM에는 데이타 최신성에 대한 문제가 있다. GPT-4 처럼 최신의 강력한 모델이라도 최신의 사건/이벤트에 대해서는 알 수 있을 방법이 없다.
LLM이 보는 세상은 특정 시점에 고정이 되어 있다. LLM은 LLM이 훈련을 받은 데이타에 들어있는 그 세계에 대해서만 알고 있다.
그래서 최신 정보를 다루어야 하거나 특정 데이타셋을 이용해야 하는 경우에는 곤란한 경우가 발생한다. 예를 들어 LLM을 통해서 회사 내부 자료(documents)에 대한 뭔가를 하는 애플리케이션을 만들 어야 하는 경우를 말한다.

제일 먼저 마주하는 어려운 점은 LLM에게 이런 자료를 추가를 하는 부분이다. LLM이 이런 자료에 대해서 훈련을 시키려고 시도를 할 수도 있지만 이것은 시간도 많이 걸리고 비용이 많이 드는 작업이다. 더군다나, 회사의 자료들도 계속해서 새로운 것들이 만들어진다. 그럴 때마다 훈련을 시킬 수도 없을 뿐더러 매분 매초마다 자료가 추가되는 상황에서 훈련을 시킨 후에 LLM을 이용한다는 것은 비효율적일 뿐만아니라 사실상 불가능하다.

그러면 이 문제를 어떻게 처리할 것인가? 여기에 <i>retrieval augmentation</i>가 필요하다. 이 기술은 외부 지식으로부터 관련된 정보를 찾아내서 그 정보를 LLM에게 전달해 주는 역할을 한다.

외부 knowledge base는 LLM의 훈련 데이타의 범위를 벗어나 세계를 쳐다볼 수 있는 '창(window)'의 역할을 한다. 이 챕터에서는 Langchain에서 LLM이 retrieval augmentation을 구현하는 것에 대해서 알아본다.

## Creating the Knowledge Base

LLM의 관점에서 지식(knowledge)는 두가지 타입이 있다. 하나는 <i>parametric knowledge[^1]</i>라고 하는 것인데 LLM이 훈련을 받는 과정에 들어온 모든 정보를 말한다. 따라서 이 정보들은 LLM이 세상을 배울 때 사용했던 그 순간의 투영(frozen snapshot)이다.

두번째 타입의 지식은 <i>source knowledge</i>라고 한다. LLM에게 인풋 프롬프트를 통해서 공급되는 정보를 말한다. 우리가 <i>retrieval augmentation</i>에 대해서 이야기 할 때에는 이것은 전부다 source knowledge이다.

[^1]Parameter라는 단어는 LLM의 기본이 되는 neural network 알고리즘은 모든 것이 함수이고 그 함수의 복잡도는 파라미터의 개수에 의해 결정된다. 수십억개의 파라미터를 찾아내는 과정을 훈련을 하는 것이라고 생각할 수 있다. 이 파라미터들이 결국은 지식이라는 뜻에서 'parametric knowledge'라고 언급된 것 같다.

### Getting Data for our Knowledge Base

LLM이 관련되어 있는 knowledge base에 접근을 할 수 있도록 하려면 먼저 knowledge base를 만들어야 한다.

먼저 knowledge base를 위한 데이타셋부터 준비하자. 만약 코딩을 도와주는 LLM을 위해서라면 코드와 관련된 문서(매뉴얼)이 데이타셋일 수 있고, 사내 챗본을 위해서라면 회사 문서일 수 있다.

여기서는 위키피디아의 일부 데이타를 예제로 사용한다. 위키피디아의 데이타셋은 Hugging Face의 데이타셋을 이용하면 된다. Hugging Face의 데이타셋에 대해서는 https://huggingface.co/docs/datasets/index 에서 좀 더 살펴볼 필요가 있다.

In [1]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')
data

/Users/minsoojo/Library/Caches/pypoetry/virtualenvs/pinecone-ai-handbook-m0t52s90-py3.10/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})

In [2]:
data[6]

{'id': '13',
 'url': 'https://simple.wikipedia.org/wiki/Alan%20Turing',
 'title': 'Alan Turing',
 'text': 'Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dub

대부분의 데이타셋은 긴 텍스트(문장)을 포함하고 있는 행(record)으로 되어 있다. 그래서 우리가 첫번째로 해야할 것은 이 긴 문장을 좀 더 간결한 chunk(문장 덩어리)로 만드는 사전 처리 파이프라인을 만드는 것이다.

### Creating Chunks

긴 문장을 작은 chunk로 만드는 것은 여러가지 이유로 반드시 필요한 작업이다.
- <i>임베딩[^1] 정확성(embedding accuracy)</i>를 향상해야 한다. 임베딩의 정확도를 향상시킨다는 것은 파이프라인에서 나오는 결과의 연관성을 높이는 길이다.
- LLM에 source knowledge로써 입력해 주는 텍스트의 양을 줄인다. 인풋 크기를 제한하는 것은 LLM의 능력을 향상시켜서 인스트럭션을 잘 따르게 하고, 비용을 줄이며, 답변 속도를 빠르게 할 수 있다.
- chunk의 크기가 아주 길면 context window 최대치를 넘어갈 수도 있다. 따라서 chunk를 더 잘게 나누게 되면 긴 문서를 knowledge base에 추가할 수 있다.

[^1] 임베딩에 대해서도 자세히 알 필요는 없다. 다만 문장덩어리(chunk)를 벡터(행열)로 만들어서 서로 다른 chunk끼리의 가까운 정도를 통해서 연관된 문장/단어를 찾아내는 것이 필요한데 이렇게 벡터화 하는 것을 임베딩, 벡터임베딩 등으로 이야기 한다.

Chunk를 만들 때에는 먼저 텍스트의 길이를 측정을 해야 한다. 측정 단위는 'token'이다. LLM이 단어의 갯수나 알파벳의 갯수로 측정을 하지 않기 때문이다. Token은 일반적으로 단어 혹은 단어의 부분(sub-word)를 기반으로 하지만 가변적이다. Token은 <i>tokenizer</i>를 통해서 생성이 된다. Tokenizer로는 toktoken 라이브러리를 사용할 것이다.

In [3]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

위 tokenizer를 통해서 token의 갯수를 세는 함수를 하나 만든다.

In [4]:
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

28

함수가 마련되었기 때문에 이제 Langchain의 <b>RecursiveCharacterTextSplitter</b> 객체를 초기화할 수 있다. 이 객체는 우리가 지정하는 chunk_size 파라미터보다 더 길지 않도록 문장을 chunk로 나누게 된다.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

# 아래는 chunk를 한 번 나누어 보는 예제
chunks = text_splitter.split_text(data[6]['text'])[:3]
chunks

['Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathison Turing, latter son o

실제 위키피디어 문서인 https://simple.wikipedia.org/wiki/Alan_Turing 에 있는 내용을 chunk 길이가 400이 넘지 않도록 나눈 것이다.

In [9]:
tiktoken_len(chunks[0]),tiktoken_len(chunks[1]),tiktoken_len(chunks[2])

(397, 304, 370)

<b>text_splitter</b>를 이용해서 문장을 적절히(400이하) chunk로 나누게 되었다. 이제는 이 기능을 이용해서 인덱스를 만드는 작업을 할 것이다. 인덱스를 만들 때 <i>embedding</i>이라는 과정을 보게 된다.

### Creating Embeddings

벡터 임베딩이라는 과정은 LLM이 연관된 컨텍스트를 꺼내는 데 있어서 가장 핵심적인 부분이다. knowledge base에 저장을 하고자 하는 문장을 chunk로 만들어서 이를 벡터임베딩으로 인코딩을 하게 된다.

임베딩(벡터임베딩, 혹은 벡터)은 각 chunk의 의미를 '숫자로 표현'한 것이다. 임베딩을 수행하는 것 또한 AI 언어모델을 이용을 하게 된다. AI 언어모델은 사람이 이해하는 문장(human-readable text)을 AI가 이해하는 임베딩(AI-readable embeddings)로 번역을 하는 것이다.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F994dbeafd67392ff7e1d25125b96845577b5b6a3-2457x904.png&w=3840&q=75)

그 다음 임베딩된 결과를 벡터 데이타베이스에 저장을 하게 된다. 벡터 데이터베이스는 임베딩 사이의 거리계산을 통해서 비슷한 의미를 가진 chunk를 찾을 수 있는 기능을 가지고 있다.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F1fc2d4ff7f25e791456cad9d2b76f46d312dba24-1775x974.png&w=1920&q=75)



In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(
    model=model_name
)

In [18]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

위의 예제에서, 우리가 2개의 chunk를 던졌으니 2개의 임베딩을 받았다. 각 임베딩은 1536 차원(1536-dimensional)의 벡터이다. 이는 <b>text-embedding-ada-002</b>의 출력차원이다.

자 이제 우리는 데이타셋, 텍스트 분리자(text splitter), 임베딩모델을 준비했다. 이제 knowledge base를 만들 차례이다.

### Vector Database

벡터 DB는 방대한 양의 데이타(records)중에서 비슷한 임베딩을 검색/입력/수정/삭제(CRUD)를 할 수 있도록 해주는 knowledge base이다.
여기서는 Pinecone의 벡터DB를 사용하려한다. 이를 위해서는 pinecone에서 무료 API Key를 획득해야 한다. 그 후 DB 인덱스를 초기화 해야 한다.

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

index_name = 'langchain-retrieval-augmentation'

#아래는 필요 없음.
#pinecone.init(
#    environment="default"
#)

pc = Pinecone()

# index를 만든다. ElasticSearch처럼 index는 collection이며 table이다.
pc.create_index(
    name=index_name,
    metric='dotproduct',
    dimension=len(res[0]),
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

테이블을 만들었으니 이제 데이타를 밀어 넣어서 실제 인덱스를 만들자. ID를 생성하고, 임베딩과 메타데이타를 만들어서 넣어야 한다. 시간이 많이 걸리는 작업이기 때문에 아래와 같이 배치를 통해서 좀 더 빠르게 할 수 있다.

In [26]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit=100

texts=[]
metadatas=[]

for i, record in enumerate(tqdm(data)):
    # 먼저 메타데이타를 만든다.
    metadata = {
        'wiki-id': str(record['id']),
        'source': record['url'],
        'title': record['title']
    }
    # 한 행(record)에서 chunk를 만든다.
    record_texts = text_splitter.split_text(record['text'])
    # 각각의 chunk에 대해서 메타데이타(dictionary 타입)을 만든다.
    record_metadatas=[{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]

    # 현재의 batch에다가 추가한다.
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # batch 개수에 도달하면 vector DB에 밀어 넣는다.
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts=[]
        metadatas=[]

100%|██████████| 10000/10000 [15:17<00:00, 10.90it/s] 


In [25]:
index=pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 27635}},
 'total_vector_count': 27635}

우리가 생성한 인덱스에는 27k 정도의 데이타(records)가 들어있다. 크지 않은 숫자이지만 샘플로써는 사용할 수 있을 정도이다.

## LangChain Vector Store and Querying

지금까지는 LangChain과는 전혀 상관 없이 인덱스를 구축했다. 지금 이 작업은 데이타를 가져와서->스플리터를 통해 chunk를 만들고->임베딩을 계산해서->Vector DB에 입력하는 간단한 과정이고, Pinecone 라이브러리를 직접 사용하는 것이 빠르기 때문에 LangChain과는 상관없이 했다. 이제는 다시 LangChain으로 돌아갈 때이다.

In [27]:
from langchain.vectorstores import Pinecone

text_field = "text"

index = pc.Index(index_name)

vectorstore=Pinecone(
    index, embed.embed_query, text_field
)

/Users/minsoojo/Library/Caches/pypoetry/virtualenvs/pinecone-ai-handbook-m0t52s90-py3.10/lib/python3.10/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


일단 LLM 없이 <b>유사도 검색(similarity search)</b> 메쏘드를 사용해서 vector DB에서부터 곧바로 chunk를 찾아내는 query를 하는 것을 살펴보자.

In [29]:
query = "who was Benito Mussolini?"

vectorstore.similarity_search(
    query,
    k=6
)

[Document(page_content='Benito Amilcare Andrea Mussolini KSMOM GCTE (29 July 1883 – 28 April 1945) was an Italian politician and journalist. He was also the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party.\n\nBiography\n\nEarly life\nBenito Mussolini was named after Benito Juarez, a Mexican opponent of the political power of the Roman Catholic Church, by his anticlerical (a person who opposes the political interference of the Roman Catholic Church in secular affairs) father. Mussolini\'s father was a blacksmith. Before being involved in politics, Mussolini was a newspaper editor (where he learned all his propaganda skills) and elementary school teacher.\n\nAt first, Mussolini was a socialist, but when he wanted Italy to join the First World War, he was thrown out of the socialist party. He \'invented\' a new ideology, Fascism, much out of Nationalist\xa0and Conservative views.\n\nRise to power and becoming dictator\nIn 1922, he took power b

pinecone에 index를 두 번 만들었기 때문에 위와 같이 같은 문장들이 두 번씩 나온다(역자 실수).
어쨌던 이 결과 문장들을 살펴보면 모두 다 질문과 연관된 결과들이다. 즉 제대로 검색을 해오고 있다. 이제 우리가 할 일은 LLM을 붙여서 이 정보를 이용해 답변을 만들 차례이다.

### Generative Question Answering

generative question-answering(GQA)을 할 때에는 우리는 LLM에게 질문을 할 때, 우리가 만든 knowledge base에서 관련된 정보를 꺼내서 인스트럭션을 하게 된다. 이 작업은 <b>RetrievalQA</b> 체인을 통해서 구현하게 된다.

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI()
llm.temperature=0.0

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [31]:
qa.run(query)

'Benito Mussolini was an Italian politician, journalist, and the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party and played a significant role in the rise of fascism in Italy. Mussolini was known for his authoritarian rule and alliance with Adolf Hitler during World War II.'

어떠한 과정을 통해서 이러한 답이 나오는 지에 대해서는 LangSmith console을 살펴보면 자세히 나온다. Retriever를 통해 qa가 만든 prompt는 아래와 같다.

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Benito Amilcare Andrea Mussolini KSMOM GCTE (29 July 1883 – 28 April 1945) was an Italian politician and journalist. He was also the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party.

Biography

Early life
Benito Mussolini was named after Benito Juarez, a Mexican opponent of the political power of the Roman Catholic Church, by his anticlerical (a person who opposes the political interference of the Roman Catholic Church in secular affairs) father. Mussolini's father was a blacksmith. Before being involved in politics, Mussolini was a newspaper editor (where he learned all his propaganda skills) and elementary school teacher.

At first, Mussolini was a socialist, but when he wanted Italy to join the First World War, he was thrown out of the socialist party. He 'invented' a new ideology, Fascism, much out of Nationalist and Conservative views.

Rise to power and becoming dictator
In 1922, he took power by having a large group of men, "Black Shirts," march on Rome and threaten to take over the government. King Vittorio Emanuele III gave in, allowed him to form a government, and made him prime minister. In the following five years, he gained power, and in 1927 created the OVRA, his personal secret police force. Using the agency to arrest, scare, or murder people against his regime, Mussolini was dictator of Italy by the end of 1927. Only the King and his own Fascist party could challenge his power.

Benito Amilcare Andrea Mussolini KSMOM GCTE (29 July 1883 – 28 April 1945) was an Italian politician and journalist. He was also the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party.

Biography

Early life
Benito Mussolini was named after Benito Juarez, a Mexican opponent of the political power of the Roman Catholic Church, by his anticlerical (a person who opposes the political interference of the Roman Catholic Church in secular affairs) father. Mussolini's father was a blacksmith. Before being involved in politics, Mussolini was a newspaper editor (where he learned all his propaganda skills) and elementary school teacher.

At first, Mussolini was a socialist, but when he wanted Italy to join the First World War, he was thrown out of the socialist party. He 'invented' a new ideology, Fascism, much out of Nationalist and Conservative views.

Rise to power and becoming dictator
In 1922, he took power by having a large group of men, "Black Shirts," march on Rome and threaten to take over the government. King Vittorio Emanuele III gave in, allowed him to form a government, and made him prime minister. In the following five years, he gained power, and in 1927 created the OVRA, his personal secret police force. Using the agency to arrest, scare, or murder people against his regime, Mussolini was dictator of Italy by the end of 1927. Only the King and his own Fascist party could challenge his power.

Fascism as practiced by Mussolini
Mussolini's form of Fascism, "Italian Fascism"- unlike Nazism, the racist ideology that Adolf Hitler followed- was different and less destructive than Hitler's. Although a believer in the superiority of the Italian nation and national unity, Mussolini, unlike Hitler, is quoted "Race? It is a feeling, not a reality. Nothing will ever make me believe that biologically pure races can be shown to exist today".

Mussolini wanted Italy to become a new Roman Empire. In 1923, he attacked the island of Corfu, and in 1924, he occupied the city state of Fiume. In 1935, he attacked the African country Abyssinia (now called Ethiopia). His forces occupied it in 1936. Italy was thrown out of the League of Nations because of this aggression. In 1939, he occupied the country Albania. In 1936, Mussolini signed an alliance with Adolf Hitler, the dictator of Germany.

Fall from power and death
In 1940, he sent Italy into the Second World War on the side of the Axis countries. Mussolini attacked Greece, but he failed to conquer it. In 1943, the Allies landed in Southern Italy. The Fascist party and King Vittorio Emanuel III deposed Mussolini and put him in jail, but he was set free by the Germans, who made him ruler of the Italian Social Republic puppet state which was in a small part of Central Italy. When the war was almost over, Mussolini tried to escape to Switzerland with his mistress, Clara Petacci, but they were both captured and shot by partisans. Mussolini's dead body was hanged upside-down, together with his mistress and some of Mussolini's helpers, on a pole at a gas station in the village of Millan, which is near the border  between Italy and Switzerland.

Fascism as practiced by Mussolini
Mussolini's form of Fascism, "Italian Fascism"- unlike Nazism, the racist ideology that Adolf Hitler followed- was different and less destructive than Hitler's. Although a believer in the superiority of the Italian nation and national unity, Mussolini, unlike Hitler, is quoted "Race? It is a feeling, not a reality. Nothing will ever make me believe that biologically pure races can be shown to exist today".

Mussolini wanted Italy to become a new Roman Empire. In 1923, he attacked the island of Corfu, and in 1924, he occupied the city state of Fiume. In 1935, he attacked the African country Abyssinia (now called Ethiopia). His forces occupied it in 1936. Italy was thrown out of the League of Nations because of this aggression. In 1939, he occupied the country Albania. In 1936, Mussolini signed an alliance with Adolf Hitler, the dictator of Germany.

Fall from power and death
In 1940, he sent Italy into the Second World War on the side of the Axis countries. Mussolini attacked Greece, but he failed to conquer it. In 1943, the Allies landed in Southern Italy. The Fascist party and King Vittorio Emanuel III deposed Mussolini and put him in jail, but he was set free by the Germans, who made him ruler of the Italian Social Republic puppet state which was in a small part of Central Italy. When the war was almost over, Mussolini tried to escape to Switzerland with his mistress, Clara Petacci, but they were both captured and shot by partisans. Mussolini's dead body was hanged upside-down, together with his mistress and some of Mussolini's helpers, on a pole at a gas station in the village of Millan, which is near the border  between Italy and Switzerland.|


하지만 이렇게 하더라도 완벽하게 hallucination을 막는 것은 안된다. 환각은 항상 발생할 수 있으며 완벽하게 막는 것은 쉽지 않다. 하지만 나오는 답변에 대해서 좀더 신뢰성을 향상시킬 수 있는 방법은 또 여러가지가 있다. 그 중 좀 효과적인 방법은 '근거'를 답변에다가 제시하라고 하는 방법이다. 이렇게 하면 사용자는 답변에 나오는 정보가 어디에서 나왔는 지를 알 수 있기 때문에 판단을 할 수 있다. LangChain에서는 앞서 사용한 <b>RetrievalQA</b>대신 <b>RetrievalQAWithSourceChain</b>를 통해서 할 수 있다.

In [36]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_source = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa_with_source(query)

{'question': 'who was Benito Mussolini?',
 'answer': 'Benito Mussolini was an Italian politician and journalist who served as the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party. Mussolini was known for his form of Fascism, "Italian Fascism," which aimed to make Italy a new Roman Empire. He allied with Adolf Hitler and led Italy into the Second World War on the side of the Axis countries. Mussolini was eventually deposed and captured by partisans, leading to his execution. \n',
 'sources': 'https://simple.wikipedia.org/wiki/Benito%20Mussolini'}

지금까지 vectorstore를 knowledge base로 하여 이를 이용해 source knowledge를 꺼집어 내서 이를 LLM 답변의 기반을 만드는 방법에 대해서 알아보았다. source knowledge를 최신화 하고 근거를 제시하게 함으로써 답변의 신뢰도를 높일 수 있다.

Bing의 AI 검색이나 Google의 Bard, 그리고 다양한 ChatGPT plugin 같은데서 LLM이 knowledge base를 사용하는 것을 알 수 있다. 미래의 LLM은 더욱더 고성능의 확장가능하면서 신뢰성 있는 knowledge base와 결합된 형태로 사용이 될 것이다.